<a href="https://colab.research.google.com/github/kluless13/paper2/blob/main/fishtally_demo_multiple_lines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing Dependencies

In [5]:
!nvidia-smi

Fri Dec 15 08:04:22 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
import os
HOME = os.getcwd()
print(HOME)

from IPython import display
display.clear_output()
from ultralytics import YOLO
from IPython.display import display, Image
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.106 🚀 Python-3.10.12 torch-2.1.2+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 34.2/166.8 GB disk)


In [6]:
# Load the YOLOv8 model
model = YOLO('/content/multiclass-wts.pt')

In [7]:
CLASS_NAMES_DICT = model.model.names
print(CLASS_NAMES_DICT)

{0: 'Blue-Tang', 1: 'Orange-Clown', 2: 'Three-Striped-Damselfish', 3: 'Yellow-Tang'}


## FishTally - 2 lines

In [1]:
import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.14.0


In [8]:
import cv2
from collections import defaultdict
import supervision as sv
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('/content/multiclass-wts.pt')

# Set up video capture
cap = cv2.VideoCapture("/content/Tang.mp4")

# Define first line coordinates
START = sv.Point(100,300)
END = sv.Point(600,700)

# Define second line coordinates
START2 = sv.Point(600,100)
END2 = sv.Point(1000,600)

# Store the track history
track_history = defaultdict(lambda: [])

# Create a dictionary to keep track of objects that have crossed the line
crossed_objects = {}

# Dictionary for second line
crossed_objects2 = {}

# Open a video sink for the output video
video_info = sv.VideoInfo.from_video_path("/content/Tang.mp4")
with sv.VideoSink("fishtally_multi_line_1.mp4", video_info) as sink:

    while cap.isOpened():
        success, frame = cap.read()

        if success:
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame, classes=[3], persist=True, save=True, tracker="bytetrack.yaml")

            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            detections = sv.Detections.from_yolov8(results[0])

            # Plot the tracks and count objects crossing the line
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 30 tracks for 30 frames
                    track.pop(0)

                # Check if the object crosses the line
                if START.x < x < END.x and min(START.y, END.y) < y < max(START.y, END.y):
                    if track_id not in crossed_objects:
                        crossed_objects[track_id] = True
                # Check if objects have crossed line 2
                if START2.x < x < END2.x and min(START2.y, END2.y) < y < max(START2.y, END2.y):
                    if track_id not in crossed_objects2:
                        crossed_objects2[track_id] = True

                    # Annotate the object as it crosses the line
                    cv2.rectangle(annotated_frame, (int(x - w / 2), int(y - h / 2)), (int(x + w / 2), int(y + h / 2)), (0, 255, 0), 2)

            # Draw the lines on the frame
            cv2.line(annotated_frame, (START.x, START.y), (END.x, END.y), (0, 255, 0), 2)
            cv2.line(annotated_frame, (START2.x, START2.y), (END2.x, END2.y), (0, 255, 0), 2)

            # Write the count of objects on each frame
            count_text = f"Objects crossed: {len(crossed_objects)}"
            count_text2 = f"Objects crossed: {len(crossed_objects2)}"
            cv2.putText(annotated_frame, count_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(annotated_frame, count_text2, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Write the frame with annotations to the output video
            sink.write_frame(annotated_frame)
        else:
            break

# Release the video capture
cap.release()

requirements: Ultralytics requirement "lap>=0.4" not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for lap: filename=lap-0.4.0-cp310-cp310-linux_x86_64.whl size=1628950 sha256=17fc32adfbeb982d963d53e6d6b83e76593df6a3a6564541393b135939ec25bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-ig1npyxi/wheels/00/42/2e/9dfe19270eea279d79e84767ff0d7b8082c3bf776cad00e83d
Successfully built lap

requirements: 1 package updated per ['lap>=0.4']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 480x800 13 Yellow-Tangs, 136.4ms
Speed: 20.7ms preprocess, 136.4ms inference, 553.5ms postprocess per image at shape (1, 3, 800, 800)
Results saved to runs/detect/track
<ipython-input-8-808f1c5122a6>:46: DeprecationWarning: from_yolov8 is deprecated: 
        This method is deprecated and remov

## FishTally - multiple lines

In [12]:
import cv2
from collections import defaultdict
import supervision as sv
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('/content/multiclass-wts.pt')

# Set up video capture
cap = cv2.VideoCapture("/content/Tang.mp4")

START = sv.Point(100,100)
END = sv.Point(200,700)
START2 = sv.Point(300,400)
END2 = sv.Point(1000,400)
START3 = sv.Point(1000,100)
END3 = sv.Point(1200,600)

# Store the track history
track_history = defaultdict(lambda: [])

crossed_objects = {}
crossed_objects2 = {}
crossed_objects3 = {}

# Open a video sink for the output video
video_info = sv.VideoInfo.from_video_path("/content/Tang.mp4")
with sv.VideoSink("fishtally_multi_line_2.mp4", video_info) as sink:

    while cap.isOpened():
        success, frame = cap.read()

        if success:
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame, classes=[3], persist=True, save=True, tracker="bytetrack.yaml")

            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            detections = sv.Detections.from_yolov8(results[0])

            # Plot the tracks and count objects crossing the line
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 30 tracks for 30 frames
                    track.pop(0)

                if START.x < x < END.x and abs(y - START.y) < 5:
                    if track_id not in crossed_objects:
                        crossed_objects[track_id] = True

                if START2.x < x < END2.x and min(START2.y, END2.y) < y < max(START2.y, END2.y):
                    if track_id not in crossed_objects2:
                        crossed_objects2[track_id] = True

                if START3.x < x < END3.x and min(START3.y, END3.y) < y < max(START3.y, END3.y):
                    if track_id not in crossed_objects3:
                        crossed_objects3[track_id] = True

                    # Annotate the object as it crosses the line
                    cv2.rectangle(annotated_frame, (int(x - w / 2), int(y - h / 2)), (int(x + w / 2), int(y + h / 2)), (0, 255, 0), 2)

            cv2.line(annotated_frame, (START.x, START.y), (END.x, END.y), (0, 255, 0), 2)
            cv2.line(annotated_frame, (START2.x, START2.y), (END2.x, END2.y), (0, 255, 0), 2)
            cv2.line(annotated_frame, (START3.x, START3.y), (END3.x, END3.y), (0, 255, 0), 2)

            # Write the count of objects on each frame
            count_text = f"Objects crossed: {len(crossed_objects)}"
            count_text2 = f"Objects crossed: {len(crossed_objects2)}"
            count_text3 = f"Objects crossed: {len(crossed_objects3)}"
            cv2.putText(annotated_frame, count_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(annotated_frame, count_text2, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(annotated_frame, count_text3, (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Write the frame with annotations to the output video
            sink.write_frame(annotated_frame)
        else:
            break

# Release the video capture
cap.release()


0: 480x800 13 Yellow-Tangs, 36.8ms
Speed: 3.7ms preprocess, 36.8ms inference, 2.5ms postprocess per image at shape (1, 3, 800, 800)
Results saved to runs/detect/track3
<ipython-input-12-f681d4d83da3>:43: DeprecationWarning: from_yolov8 is deprecated: 
        This method is deprecated and removed in 0.15.0 release.
        Use sv.Detections.from_ultralytics() instead as it is more generic and
        can be used for detections from any ultralytics.engine.results.Results Object
        
  detections = sv.Detections.from_yolov8(results[0])

0: 480x800 13 Yellow-Tangs, 35.9ms
Speed: 4.5ms preprocess, 35.9ms inference, 1.8ms postprocess per image at shape (1, 3, 800, 800)
Results saved to runs/detect/track3

0: 480x800 13 Yellow-Tangs, 35.9ms
Speed: 4.4ms preprocess, 35.9ms inference, 1.8ms postprocess per image at shape (1, 3, 800, 800)
Results saved to runs/detect/track3

0: 480x800 13 Yellow-Tangs, 35.9ms
Speed: 4.4ms preprocess, 35.9ms inference, 1.7ms postprocess per image at shape (